In [5]:
#!/usr/bin/env python3
"""
apply_color_correction_to_taylor_FIXED_v2.py

Versión corregida - Usa los nombres correctos de columnas (MAG_FXXX_3)
"""

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
import os
warnings.filterwarnings('ignore')

def debug_dataframe_info(df, name):
    """Función de debug para verificar el contenido del DataFrame"""
    print(f"\n🔍 DEBUG - Información de {name}:")
    print(f"  Forma: {df.shape}")
    print(f"  Columnas relacionadas con MAG: {[col for col in df.columns if 'MAG_' in col]}")
    print("-" * 50)

def load_color_transformations(transformation_file='color_transformations.csv'):
    """Carga las ecuaciones de transformación derivadas"""
    if not os.path.exists(transformation_file):
        print(f"❌ ERROR: No se encuentra el archivo {transformation_file}")
        return None
    
    trans_df = pd.read_csv(transformation_file)
    print("Transformaciones cargadas:")
    print(trans_df[['field', 'splus_filter', 'transformation_equation', 'rms']].to_string(index=False))
    return trans_df

def calculate_approximate_bp_rp(taylor_df):
    """
    Calcula una aproximación de (BP-RP) usando los filtros de Taylor.
    """
    # Verificar si tenemos las columnas necesarias
    if 'gmag' in taylor_df.columns and 'imag' in taylor_df.columns:
        bp_rp_approx = (taylor_df['gmag'] - taylor_df['imag']) - 0.1
        print("✅ Color (BP-RP) aproximado calculado como (g-i) - 0.1")
        return bp_rp_approx
    else:
        print("⚠️  No se pueden calcular colores, usando valor por defecto para GCs viejos")
        # Para cúmulos globulares viejos, (BP-RP) típico ~ 0.8-1.2
        return np.ones(len(taylor_df)) * 1.0

def apply_gaia_based_corrections(taylor_df, trans_df, aperture=3):
    """
    Aplica correcciones basadas en las transformaciones Gaia-SPLUS
    """
    if trans_df is None:
        print("❌ ERROR: No hay transformaciones para aplicar")
        return taylor_df, None
    
    # Usar el promedio de las transformaciones de ambos campos
    avg_trans = trans_df.groupby('splus_filter').agg({
        'constant_term': 'mean',
        'color_term': 'mean',
        'rms': 'mean'
    }).reset_index()
    
    print("\n📐 Transformaciones promedio aplicadas:")
    for _, row in avg_trans.iterrows():
        print(f"  {row['splus_filter']}: const={row['constant_term']:.4f}, color_term={row['color_term']:.4f}")
    
    # Calcular color aproximado
    bp_rp_approx = calculate_approximate_bp_rp(taylor_df)
    
    # Aplicar correcciones a cada filtro
    corrected_df = taylor_df.copy()
    
    print("\n🔧 Aplicando correcciones a cada filtro:")
    for _, trans in avg_trans.iterrows():
        splus_filter = trans['splus_filter']
        # ✅ CORRECCIÓN: Usar el nombre correcto con prefijo MAG_
        splus_col = f'MAG_{splus_filter}_{aperture}'
        
        print(f"  Procesando {splus_col}...")
        
        if splus_col not in corrected_df.columns:
            print(f"  ⚠️  Columna {splus_col} no encontrada")
            continue
        
        # Verificar que tenemos datos no nulos y que no sean 99.0 (valores inválidos)
        valid_mask = (corrected_df[splus_col].notna() & 
                     (corrected_df[splus_col] < 90) & 
                     (corrected_df[splus_col] > 10))
        
        if valid_mask.sum() == 0:
            print(f"  ⚠️  Columna {splus_col} no tiene datos válidos (todos nulos o 99.0)")
            continue
        
        # Calcular corrección: correction = constant_term + color_term * (BP-RP)
        correction = trans['constant_term'] + trans['color_term'] * bp_rp_approx
        
        # Aplicar corrección: mag_corrected = mag_original - correction
        # ✅ CORRECCIÓN: Usar nombres consistentes con prefijo MAG_
        corrected_col = f'MAG_{splus_filter}_corrected_{aperture}'
        corrected_df[corrected_col] = corrected_df[splus_col] - correction
        
        print(f"  ✅ Aplicada corrección a {splus_col} -> {corrected_col}")
        print(f"     Datos válidos: {valid_mask.sum()}/{len(corrected_df)}")
        print(f"     Rango original: [{corrected_df.loc[valid_mask, splus_col].min():.2f}, {corrected_df.loc[valid_mask, splus_col].max():.2f}]")
        print(f"     Rango corregido: [{corrected_df.loc[valid_mask, corrected_col].min():.2f}, {corrected_df.loc[valid_mask, corrected_col].max():.2f}]")
        
        # También calcular incertidumbre propagada
        mag_err_col = f'MAGERR_{splus_filter}_{aperture}'
        if mag_err_col in corrected_df.columns:
            corrected_err_col = f'MAG_{splus_filter}_corrected_err_{aperture}'
            corrected_df[corrected_err_col] = corrected_df[mag_err_col]
    
    return corrected_df, bp_rp_approx

def analyze_corrected_photometry(original_df, corrected_df, aperture=3):
    """
    Analiza la mejora después de aplicar las correcciones de color
    """
    print(f"\n📊 Iniciando análisis de fotometría corregida (aperture={aperture})")
    
    # ✅ CORRECCIÓN: Usar los nombres correctos de columnas
    filter_correspondences = [
        ('MAG_F378', 'umag', 'F378 vs u-band'),
        ('MAG_F395', 'umag', 'F395 vs u-band'),
        ('MAG_F410', 'gmag', 'F410 vs g-band'),
        ('MAG_F430', 'gmag', 'F430 vs g-band'),
        ('MAG_F515', 'gmag', 'F515 vs g-band'),
        ('MAG_F660', 'rmag', 'F660 vs r-band'),
        ('MAG_F861', 'imag', 'F861 vs i-band')
    ]
    
    # Verificar qué columnas tenemos disponibles
    print("🔍 Verificando columnas disponibles:")
    available_splus = [col for col in original_df.columns if f'_{aperture}' in col and 'MAG_' in col and 'corrected' not in col]
    available_taylor = [col for col in original_df.columns if 'mag' in col and col not in available_splus]
    
    print(f"  Columnas SPLUS disponibles: {available_splus}")
    print(f"  Columnas Taylor disponibles: {available_taylor}")
    
    # Crear figura comparativa
    fig, axes = plt.subplots(4, 2, figsize=(15, 20))
    axes = axes.ravel()
    
    results = []
    plots_created = 0
    
    for i, (splus_filter, taylor_filter, title) in enumerate(filter_correspondences):
        if i >= len(axes):
            break
            
        splus_col = f'{splus_filter}_{aperture}'
        corrected_col = f'{splus_filter}_corrected_{aperture}'
        
        print(f"\n  Analizando {splus_filter} vs {taylor_filter}:")
        print(f"    SPLUS col: {splus_col} (¿existe? {splus_col in original_df.columns})")
        print(f"    Taylor col: {taylor_filter} (¿existe? {taylor_filter in original_df.columns})")
        print(f"    Corrected col: {corrected_col} (¿existe? {corrected_col in corrected_df.columns})")
        
        if splus_col not in original_df.columns or taylor_filter not in original_df.columns:
            print(f"    ⚠️  Saltando - columnas necesarias no encontradas")
            continue
        
        # Datos originales - filtrar valores inválidos (99.0)
        valid_original = (
            original_df[splus_col].notna() & 
            original_df[taylor_filter].notna() &
            np.isfinite(original_df[splus_col]) & 
            np.isfinite(original_df[taylor_filter]) &
            (original_df[splus_col] < 90) &  # Excluir 99.0
            (original_df[taylor_filter] < 90) &
            (original_df[splus_col] > 10) & 
            (original_df[taylor_filter] > 10)
        )
        
        valid_count_original = valid_original.sum()
        print(f"    Datos originales válidos: {valid_count_original}")
        
        if valid_count_original < 5:
            print(f"    ⚠️  Saltando - muy pocos datos originales válidos")
            continue
        
        # Calcular diferencias para datos originales
        splus_orig = original_df.loc[valid_original, splus_col]
        taylor_orig = original_df.loc[valid_original, taylor_filter]
        diff_original = splus_orig - taylor_orig
        
        # Verificar si tenemos datos corregidos
        has_correction = (corrected_col in corrected_df.columns and 
                         not corrected_df[corrected_col].isna().all())
        
        if has_correction:
            # Datos corregidos - también filtrar valores inválidos
            valid_corrected = (
                corrected_df[corrected_col].notna() & 
                corrected_df[taylor_filter].notna() &
                np.isfinite(corrected_df[corrected_col]) & 
                np.isfinite(corrected_df[taylor_filter]) &
                (corrected_df[corrected_col] < 90) &  # Excluir 99.0
                (corrected_df[taylor_filter] < 90) &
                (corrected_df[corrected_col] > 10) & 
                (corrected_df[taylor_filter] > 10)
            )
            
            valid_count_corrected = valid_corrected.sum()
            print(f"    Datos corregidos válidos: {valid_count_corrected}")
            
            if valid_count_corrected > 5:
                splus_corr = corrected_df.loc[valid_corrected, corrected_col]
                taylor_corr = corrected_df.loc[valid_corrected, taylor_filter]
                diff_corrected = splus_corr - taylor_corr
                
                # Estadísticas
                med_orig = np.median(diff_original)
                med_corr = np.median(diff_corrected)
                mad_orig = np.median(np.abs(diff_original - med_orig))
                mad_corr = np.median(np.abs(diff_corrected - med_corr))
                
                results.append({
                    'filter': splus_filter,
                    'taylor_filter': taylor_filter,
                    'n_original': len(diff_original),
                    'n_corrected': len(diff_corrected),
                    'median_original': med_orig,
                    'median_corrected': med_corr,
                    'mad_original': mad_orig,
                    'mad_corrected': mad_corr,
                    'improvement': mad_orig - mad_corr
                })
                
                # Crear subplot
                ax = axes[plots_created]
                # Original (rojo)
                ax.scatter(taylor_orig, splus_orig, alpha=0.4, s=20, c='red', 
                          label=f'Original: Δ={med_orig:.3f}')
                # Corregido (azul)
                ax.scatter(taylor_corr, splus_corr, alpha=0.6, s=20, c='blue',
                          label=f'Corregido: Δ={med_corr:.3f}')
                
                # Línea 1:1
                x_range = np.linspace(min(taylor_orig.min(), taylor_corr.min()), 
                                    max(taylor_orig.max(), taylor_corr.max()), 100)
                ax.plot(x_range, x_range, 'k--', alpha=0.5, label='1:1')
                
                ax.set_xlabel(f'Taylor {taylor_filter}')
                ax.set_ylabel(f'SPLUS {splus_filter}')
                ax.set_title(f'{title}\nMAD: {mad_orig:.3f}→{mad_corr:.3f}')
                ax.legend(fontsize=8)
                ax.grid(True, alpha=0.3)
                
                plots_created += 1
                print(f"    ✅ Gráfico creado para {splus_filter}")
            else:
                print(f"    ⚠️  No hay suficientes datos corregidos válidos")
        else:
            print(f"    ⚠️  No hay datos corregidos disponibles")
    
    # Ocultar ejes vacíos
    for j in range(plots_created, len(axes)):
        axes[j].set_visible(False)
    
    if plots_created > 0:
        plt.suptitle('Comparación SPLUS vs Taylor - Antes y Después de Correcciones de Color', 
                     fontsize=16, y=0.98)
        plt.tight_layout()
        output_plot = f'taylor_comparison_color_corrected_aper{aperture}.png'
        plt.savefig(output_plot, dpi=300, bbox_inches='tight')
        plt.close()
        print(f"✅ Gráfico guardado: {output_plot}")
    else:
        plt.close()
        print("⚠️  No se crearon gráficos - no hay datos suficientes")
    
    return pd.DataFrame(results)

def plot_improvement_summary(results_df):
    """Gráfico resumen de la mejora obtenida"""
    if results_df.empty:
        print("⚠️  No hay resultados para crear el resumen de mejora")
        return
    
    try:
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
        
        # Gráfico de offsets medianos
        x_pos = np.arange(len(results_df))
        width = 0.35
        
        ax1.bar(x_pos - width/2, results_df['median_original'], width, 
                label='Original', alpha=0.7, color='red')
        ax1.bar(x_pos + width/2, results_df['median_corrected'], width, 
                label='Corregido', alpha=0.7, color='blue')
        
        ax1.set_xlabel('Filtro')
        ax1.set_ylabel('Offset Mediano (SPLUS - Taylor)')
        ax1.set_title('Reducción de Offsets Sistemáticos')
        ax1.set_xticks(x_pos)
        ax1.set_xticklabels([f.replace('MAG_', '') for f in results_df['filter']], rotation=45)
        ax1.legend()
        ax1.grid(True, alpha=0.3)
        
        # Gráfico de mejora en MAD
        improvement = results_df['mad_original'] - results_df['mad_corrected']
        colors = ['green' if x > 0 else 'red' for x in improvement]
        
        ax2.bar(x_pos, improvement, color=colors, alpha=0.7)
        ax2.set_xlabel('Filtro')
        ax2.set_ylabel('Mejora en MAD (Original - Corregido)')
        ax2.set_title('Mejora en Precisión (MAD reducida)')
        ax2.set_xticks(x_pos)
        ax2.set_xticklabels([f.replace('MAG_', '') for f in results_df['filter']], rotation=45)
        ax2.axhline(0, color='black', linestyle='-', alpha=0.3)
        ax2.grid(True, alpha=0.3)
        
        # Anotar valores de mejora
        for i, v in enumerate(improvement):
            ax2.text(i, v + 0.001 * (1 if v >= 0 else -1), f'{v:.3f}', 
                    ha='center', va='bottom' if v >= 0 else 'top', fontsize=9)
        
        plt.tight_layout()
        plt.savefig('color_correction_improvement_summary.png', dpi=300, bbox_inches='tight')
        plt.close()
        print("✅ Gráfico de resumen de mejora guardado")
    except Exception as e:
        print(f"❌ Error creando gráfico de resumen: {e}")

def main():
    """Función principal"""
    print("🚀 INICIANDO CORRECCIÓN DE COLOR PARA COMPARACIÓN CON TAYLOR (VERSIÓN CORREGIDA v2)")
    print("="*70)
    
    # Archivos de entrada
    taylor_file = '../anac_data/Results/all_fields_gc_photometry_identical.csv'
    transformation_file = 'color_transformations.csv'
    
    # Verificar que los archivos existen
    if not os.path.exists(taylor_file):
        print(f"❌ ERROR: No se encuentra el archivo {taylor_file}")
        print("   Verifica la ruta y que el archivo existe")
        return
    
    if not os.path.exists(transformation_file):
        print(f"❌ ERROR: No se encuentra el archivo {transformation_file}")
        print("   Ejecuta primero el script de análisis de offsets")
        return
    
    # Cargar datos
    print("📊 Cargando datos de Taylor...")
    try:
        taylor_df = pd.read_csv(taylor_file)
        print(f"✅ Cargados {len(taylor_df)} cúmulos globulares")
        debug_dataframe_info(taylor_df, "Datos de Taylor originales")
    except Exception as e:
        print(f"❌ Error cargando datos de Taylor: {e}")
        return
    
    # Cargar transformaciones
    trans_df = load_color_transformations(transformation_file)
    if trans_df is None:
        return
    
    # Aplicar correcciones
    aperture = 3
    print(f"\n🎯 Aplicando correcciones con aperture={aperture}")
    corrected_df, bp_rp_approx = apply_gaia_based_corrections(taylor_df, trans_df, aperture)
    
    # Guardar datos corregidos
    output_file = 'taylor_comparison_color_corrected.csv'
    corrected_df['bp_rp_approximation'] = bp_rp_approx
    corrected_df.to_csv(output_file, index=False)
    print(f"✅ Datos corregidos guardados en: {output_file}")
    debug_dataframe_info(corrected_df, "Datos corregidos")
    
    # Analizar mejora
    print("\n📈 Analizando mejora después de correcciones...")
    results_df = analyze_corrected_photometry(taylor_df, corrected_df, aperture)
    
    if not results_df.empty:
        results_df.to_csv('color_correction_results.csv', index=False)
        print("✅ Resultados del análisis guardados en: color_correction_results.csv")
        
        # Resumen estadístico
        print("\n" + "="*70)
        print("📊 RESUMEN DE MEJORA TRAS CORRECCIONES")
        print("="*70)
        print(results_df.to_string(index=False))
        
        # Gráfico de mejora
        plot_improvement_summary(results_df)
        
        # Estadísticas generales
        avg_improvement = results_df['improvement'].mean()
        print(f"\n🎯 MEJORA PROMEDIO EN MAD: {avg_improvement:.4f} mag")
        
        if avg_improvement > 0:
            print("✅ Las correcciones de color MEJORAN la precisión")
        else:
            print("⚠️  Las correcciones no mejoran significativamente la precisión")
    else:
        print("❌ No se generaron resultados del análisis")
        print("   Esto puede deberse a:")
        print("   - No hay suficientes datos válidos")
        print("   - Las columnas necesarias no existen")
        print("   - Los datos están fuera de los rangos esperados")
    
    print("\n" + "="*70)
    print("ANÁLISIS COMPLETADO")
    print("="*70)

if __name__ == '__main__':
    main()

🚀 INICIANDO CORRECCIÓN DE COLOR PARA COMPARACIÓN CON TAYLOR (VERSIÓN CORREGIDA v2)
📊 Cargando datos de Taylor...
✅ Cargados 181 cúmulos globulares

🔍 DEBUG - Información de Datos de Taylor originales:
  Forma: (181, 206)
  Columnas relacionadas con MAG: ['MAG_F378_3', 'MAG_F378_4', 'MAG_F378_5', 'MAG_F378_6', 'MAG_F395_3', 'MAG_F395_4', 'MAG_F395_5', 'MAG_F395_6', 'MAG_F410_3', 'MAG_F410_4', 'MAG_F410_5', 'MAG_F410_6', 'MAG_F430_3', 'MAG_F430_4', 'MAG_F430_5', 'MAG_F430_6', 'MAG_F515_3', 'MAG_F515_4', 'MAG_F515_5', 'MAG_F515_6', 'MAG_F660_3', 'MAG_F660_4', 'MAG_F660_5', 'MAG_F660_6', 'MAG_F861_3', 'MAG_F861_4', 'MAG_F861_5', 'MAG_F861_6']
--------------------------------------------------
Transformaciones cargadas:
 field splus_filter                         transformation_equation      rms
CenA01         F515  SPLUS_F515 = Gaia_G + -0.3114 + 0.6946*(BP-RP) 0.036074
CenA01         F660  SPLUS_F660 = Gaia_RP + 0.2421 + 0.2448*(BP-RP) 0.021423
CenA01         F861 SPLUS_F861 = Gaia_RP + 0